ALL 7 HERDS TABLE

In [1]:
#Author Amulya J N  ojaswini2004@gmail.com

import tkinter as tk
from tkinter import ttk
import pandas as pd
import webbrowser
from datetime import datetime
import pygame
import threading
import folium
from folium.plugins import AntPath

# Function to open the HTML file in a browser
def open_html():
    webbrowser.open("E:\\Amulya B-Tech\\Code\\TC\\combined_maps.html")  # Update with the correct file path

# Function to play an audio file
def play_audio(file_path):
    pygame.mixer.init()
    pygame.mixer.music.load(file_path)
    pygame.mixer.music.set_volume(1.0)  # Set volume to maximum
    pygame.mixer.music.play()
    threading.Timer(10.0, pygame.mixer.music.stop).start()  # Stop audio after 10 seconds

# Function to check the current time and play audio if it matches the marker time
def check_time_and_play_audio():
    current_time = datetime.now().strftime('%H:%M:%S')
    if 'TC 5' in marker_details_list[0]:  # Assuming checking only the first herd for simplicity
        tc_5_time = marker_details_list[0]['TC 5'].split('Time: ')[-1]
        if tc_5_time == current_time:
            play_audio("E:\\Amulya B-Tech\\Code\\TC\\sound.mp3")  # Update with your MP3 file path
    root.after(1000, check_time_and_play_audio)  # Check every second

# Function to display maps and tables in a Tkinter window
def display_map_and_tables(lat_lon_points_list, csv_file_paths):
    global marker_details_list
    marker_details_list = []

    # Read CSV files and process data
    for i, (lat_lon_points, csv_file_path) in enumerate(zip(lat_lon_points_list, csv_file_paths)):
        data = pd.read_csv(csv_file_path)
        marker_details = {}

        # Process each CSV file
        for idx, (lat, lon) in enumerate(lat_lon_points):
            marker_label = f"TC {idx + 1}"
            row = data[(data['Latitude'] == lat) & (data['Longitude'] == lon)]
            if not row.empty:
                detail_info = row.to_dict('records')[0]
                details_str = f"{detail_info['Latitude']}, {detail_info['Longitude']}\nElevation: {detail_info['Elevation']}\nTime: {detail_info['Time']}\nDate: {detail_info['Date']}"
            else:
                details_str = "No details available"
            marker_details[marker_label] = details_str

        marker_details_list.append(marker_details)

    # Create the main window
    global root
    root = tk.Tk()
    root.title("Maps and Details Tables")
    root.geometry("1600x1000")
    root.configure(bg="#f0f8ff")

    # Create a heading
    heading = ttk.Label(root, text="Maps and TC[Trap Camera] Details", font=("Helvetica", 18, "bold"), background="#4682b4", foreground="white")
    heading.pack(pady=10, fill=tk.X)

    # Create a scrollable frame
    canvas = tk.Canvas(root, bg="#f0f8ff")
    scrollbar = ttk.Scrollbar(root, orient="vertical", command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion=canvas.bbox("all")
        )
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    # Pack the canvas and scrollbar
    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    columns = ["TC 1", "TC 2", "TC 3", "TC 4", "TC 5"]
    max_rows = 4

    # Create frames for tables within the scrollable frame
    for i in range(7):
        table_frame = ttk.Frame(scrollable_frame, padding="10", relief="sunken", borderwidth=2)
        table_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Add title for the table
        title = ttk.Label(table_frame, text=f"Herd {i+1}", font=("Helvetica", 14, "bold"), background="#f0f8ff")
        title.pack(pady=5)

        # Create the table header
        header_frame = ttk.Frame(table_frame)
        header_frame.pack(fill=tk.X)

        # Serial number and marker labels
        ttk.Label(header_frame, text="S.no", borderwidth=1, relief="solid", width=10, background="#4682b4", foreground="white").grid(row=0, column=0)
        for j, col in enumerate(columns):
            ttk.Label(header_frame, text=col, borderwidth=1, relief="solid", width=20, background="#4682b4", foreground="white").grid(row=0, column=j + 1)

        # Create the table content
        table_content_frame = ttk.Frame(table_frame)
        table_content_frame.pack()

        # Fill table content
        marker_details = marker_details_list[i]
        for row_idx in range(max_rows):
            ttk.Label(table_content_frame, text=chr(ord('A') + row_idx), borderwidth=1, relief="solid", width=10, background="#d3d3d3").grid(row=row_idx, column=0)
            for col_idx, col in enumerate(columns):
                marker_label = col
                details = marker_details.get(marker_label, "No details available")
                details_lines = details.split('\n')
                if row_idx < len(details_lines):
                    ttk.Label(table_content_frame, text=details_lines[row_idx], borderwidth=1, relief="solid", width=20, anchor='w', background="#ffffff").grid(row=row_idx, column=col_idx + 1)

    # Add a sentence below the tables
    map_info_label = ttk.Label(root, text="View the maps by clicking the button below.", font=("Helvetica", 12), background="#f0f8ff")
    map_info_label.pack(pady=10, side=tk.BOTTOM)

    # Add a button to open the HTML file
    open_button = ttk.Button(root, text="View Combined Maps", command=open_html, style="TButton")
    open_button.pack(pady=20, side=tk.BOTTOM)

    # Check time and play audio
    check_time_and_play_audio()

    # Run the Tkinter main loop
    root.mainloop()

# Function to create an individual map
def create_map(lat_lon_points, map_file_path):
    m = folium.Map(location=[lat_lon_points[0][0], lat_lon_points[0][1]], zoom_start=12)

    # Add markers
    for idx, (lat, lon) in enumerate(lat_lon_points):
        folium.Marker([lat, lon], popup=f"TC {idx + 1}").add_to(m)

    # Add animated path with arrow head
    AntPath(
        locations=[(lat, lon) for lat, lon in lat_lon_points],
        dash_array=[20, 20],
        delay=1000,
        color='blue',
        pulse_color='red',
        icon_color='green',
        pulse_scale=5
    ).add_to(m)

    # Save the map to an HTML file
    m.save(map_file_path)

# Function to create a combined map with all lat-lon points
def create_combined_map_html(lat_lon_points_list, output_path):
    combined_map = folium.Map(location=[lat_lon_points_list[0][0][0], lat_lon_points_list[0][0][1]], zoom_start=12)

    # Add markers and animated paths for each set of points
    for idx, lat_lon_points in enumerate(lat_lon_points_list):
        for point_idx, (lat, lon) in enumerate(lat_lon_points):
            folium.Marker([lat, lon], popup=f"TC {point_idx + 1} - CSV {idx + 1}").add_to(combined_map)

        AntPath(
            locations=[(lat, lon) for lat, lon in lat_lon_points],
            dash_array=[20, 20],
            delay=1000,
            color='blue',
            pulse_color='red',
            icon_color='green',
            pulse_scale=5
        ).add_to(combined_map)

    # Save combined map
    combined_map.save(output_path)

# Example usage
lat_lon_points_1 = [
    (11.67362,76.26272),
    (11.67289,76.34133),
    (11.71983,76.4164),
    (11.77865,76.40343),
    (11.86046,76.39905)
]

lat_lon_points_2 = [
    (11.92949,76.37388),
    (12.01307,76.38022),
    (12.01725,76.24367),
    (11.94741,76.14472),
    (11.99284,76.11109)
]

lat_lon_points_3 = [
    (11.98922,76.0739),
    (11.88274,76.31111),
    (11.86703,76.30613),
    (11.86703,76.30613),
    (11.80246,76.38563),
    (11.67746,76.6808)
]

lat_lon_points_4 = [
    (11.63264,76.65175),
    (11.72739,76.66303),
    (11.75069,76.62857),
    (11.75523,76.58183),
    (11.66877,76.68523)
]

lat_lon_points_5 = [
    (11.67407,76.669),
    (11.73578,76.59085),
    (11.84396,76.55204),
    (11.91993,76.54695),
    (11.96317,76.45042)
]

lat_lon_points_6 = [
    (11.99428,76.42088),
    (11.92078,76.48394),
    (11.96473,76.4553),
    (12.03044,76.40442),
    (12.01062,76.33188)
]

lat_lon_points_7 = [
    (12.04502,76.3911),
    (12.05048,76.30125),
    (12.0968,76.31187),
    (12.06391,76.23458),
    (12.00115,76.24114)
]

# List of lat-lon points for all herds
all_lat_lon_points = [
    lat_lon_points_1,
    lat_lon_points_2,
    lat_lon_points_3,
    lat_lon_points_4,
    lat_lon_points_5,
    lat_lon_points_6,
    lat_lon_points_7
]

# List of CSV file paths
csv_file_paths = [
    "E:\\Amulya B-Tech\\Code\\FINAL\\chunk_1.csv",
    "E:\\Amulya B-Tech\\Code\\FINAL\\chunk_2.csv",
    "E:\\Amulya B-Tech\\Code\\FINAL\\chunk_3.csv",
    "E:\\Amulya B-Tech\\Code\\FINAL\\chunk_4.csv",
    "E:\\Amulya B-Tech\\Code\\FINAL\\chunk_5.csv",
    "E:\\Amulya B-Tech\\Code\\FINAL\\chunk_6.csv",
    "E:\\Amulya B-Tech\\Code\\FINAL\\chunk_7.csv"
]



# Create individual maps (Optional)
for i, points in enumerate(all_lat_lon_points):
    create_map(points, f"E:\\Amulya B-Tech\\Code\\TC\\map_{i+1}.html")  # Update with the correct file paths

# Create a combined map
create_combined_map_html(all_lat_lon_points, "E:\\Amulya B-Tech\\Code\\FINAL\\combined_maps.html")

# Display the GUI with tables and maps
display_map_and_tables(all_lat_lon_points, csv_file_paths)


pygame 2.6.0 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
